## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,71.65,91,100,11.99,overcast clouds
1,1,Vestmannaeyjar,IS,63.4427,-20.2734,49.19,93,100,12.66,overcast clouds
2,2,Deogarh,IN,21.5333,84.7333,85.37,63,55,8.10,broken clouds
3,3,Leh,IN,34.1667,77.5833,50.81,34,51,7.45,broken clouds
4,4,Barrow,US,71.2906,-156.7887,21.22,86,0,5.75,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,71.65,91,100,11.99,overcast clouds
2,2,Deogarh,IN,21.5333,84.7333,85.37,63,55,8.10,broken clouds
6,6,Albany,US,42.6001,-73.9662,78.03,77,82,4.25,broken clouds
7,7,Cape Town,ZA,-33.9258,18.4232,64.35,70,0,5.01,clear sky
8,8,Lompoc,US,34.6391,-120.4579,71.92,84,75,8.05,broken clouds
9,9,Solano,US,38.2333,-121.9511,80.33,50,0,23.02,clear sky
11,11,Urucui,BR,-7.2294,-44.5561,93.31,39,99,8.19,overcast clouds
19,19,Kaeo,NZ,-35.1000,173.7833,61.99,92,94,8.66,overcast clouds
20,20,Jamestown,US,42.0970,-79.2353,78.53,64,75,8.05,broken clouds
23,23,Andover,GB,51.2083,-1.4825,64.49,86,100,10.54,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                517
City                   517
Country                512
Lat                    517
Lng                    517
Max Temp               517
Humidity               517
Cloudiness             517
Wind Speed             517
Current Description    517
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
print (clean_df)

     City_ID          City Country      Lat       Lng  Max Temp  Humidity  \
0          0      Victoria      HK  22.2855  114.1577     71.65        91   
2          2       Deogarh      IN  21.5333   84.7333     85.37        63   
6          6        Albany      US  42.6001  -73.9662     78.03        77   
7          7     Cape Town      ZA -33.9258   18.4232     64.35        70   
8          8        Lompoc      US  34.6391 -120.4579     71.92        84   
..       ...           ...     ...      ...       ...       ...       ...   
701      701         Upata      VE   8.0086  -62.3989     84.06        65   
702      702          Napa      US  38.4832 -122.3344     82.56        40   
704      704       Caucaia      BR  -3.7361  -38.6531     85.95        73   
706      706  Komsomolskiy      UZ  40.4272   71.7189     63.32        77   
707      707     Dhidhdhoo      MV   6.8833   73.1000     81.48        81   

     Cloudiness  Wind Speed Current Description  
0           100       11.

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,71.65,overcast clouds,22.2855,114.1577,
2,Deogarh,IN,85.37,broken clouds,21.5333,84.7333,
6,Albany,US,78.03,broken clouds,42.6001,-73.9662,
7,Cape Town,ZA,64.35,clear sky,-33.9258,18.4232,
8,Lompoc,US,71.92,broken clouds,34.6391,-120.4579,
9,Solano,US,80.33,clear sky,38.2333,-121.9511,
11,Urucui,BR,93.31,overcast clouds,-7.2294,-44.5561,
19,Kaeo,NZ,61.99,overcast clouds,-35.1000,173.7833,
20,Jamestown,US,78.53,broken clouds,42.0970,-79.2353,
23,Andover,GB,64.49,overcast clouds,51.2083,-1.4825,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,71.65,overcast clouds,22.2855,114.1577,Mini Central
2,Deogarh,IN,85.37,broken clouds,21.5333,84.7333,Amit Henej House
6,Albany,US,78.03,broken clouds,42.6001,-73.9662,
7,Cape Town,ZA,64.35,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
8,Lompoc,US,71.92,broken clouds,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
...,...,...,...,...,...,...,...
701,Upata,VE,84.06,overcast clouds,8.0086,-62.3989,Posada Villa Nela
702,Napa,US,82.56,few clouds,38.4832,-122.3344,Fortuna Ranch
704,Caucaia,BR,85.95,few clouds,-3.7361,-38.6531,M.O.S. Logística - Filial Fortaleza
706,Komsomolskiy,UZ,63.32,broken clouds,40.4272,71.7189,Ikathouse


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descriptio</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))